In [ ]:
# !pip install gensim
# !pip install jieba
# !pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [6]:
# %cd '/content/gdrive/MyDrive/Data_Sicience/LDS0_K282_ONLINE_LuuNguyenKhaiHoan/Project_2/Data'

In [7]:
%cd "C:\Users\khaih\Downloads\Project_2\Data"

C:\Users\khaih\Downloads\Project_2\Data


In [8]:
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import jieba
import re

In [9]:
STOP_WORD_FILE='vietnamese-stopwords.txt'

In [10]:
#LOAD wrong words
file = open('wrong-word.txt', 'r', encoding="utf8")
wrong_lst = file.read().split('\n')
file.close()

In [11]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

### Read Data

In [12]:
import pandas as pd
products= pd.read_csv("pre_products.csv",delimiter=',', index_col=0)
reviews=pd.read_csv("ReviewRaw.csv",delimiter=',')

In [13]:
products.head()

,index,item_id,rating,product_content,product_content_wt
0,0,48102821,4.0,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,vân chống nước nhạc từ sạc đầy chỉ khoảng chờ ...
1,1,52333193,4.5,Tai nghe bluetooth không dây F9 True wireless ...,không dây wireless có báo kép nhạc giờ giờ . o...
2,2,299461,4.8,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,chuột không dây hàng chính hãng thương hiệu cá...
3,3,57440329,4.7,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,robot hàng chính hãng acome sku chính hãng aco...
4,4,38458616,4.8,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,hàng chính hãng mỹ hộp sạc sạc sku và nhỏ gọn ...


### Tokenize và Xóa thêm một vài kí tự đặc biệt trong text

In [14]:
products_gem = [[text for text in x.split()] for x in products.product_content_wt]

In [15]:
products_gem_re = [[re.sub('[0-9]+','', e) for e in text] for text in products_gem] # số
products_gem_re = [[t.lower() for t in text if not t in ['', ' ', ',', '.', '...', '-',':', ';', '?', '%', '(', ')', '+', '/', 'g', 'ml']] for text in  products_gem_re] # ký tự đặc biệt
products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem_re] # stopword

In [16]:
import pickle
with open('products_gem_re.pkl', 'wb') as f:
        pickle.dump(products_gem_re, f)

### Thực hiện Gensim

In [17]:
dictionary=corpora.Dictionary(products_gem_re)

In [18]:
#number of features in dict
feature_cnt = len(dictionary.token2id)

In [19]:
#obtain corpus based on dict (dense matrix)
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [20]:
#use tf-idf model to process corpus, obtain index
tfidf = models.TfidfModel(corpus)
#tính toán sự tương quan trong ma trận thưa thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=feature_cnt)
#index là ma trận lưu s75 tương quan của các từ, từ na2p lặp nhìu lần thì điểm cao

In [21]:
products.head(3)

,index,item_id,rating,product_content,product_content_wt
0,0,48102821,4.0,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,vân chống nước nhạc từ sạc đầy chỉ khoảng chờ ...
1,1,52333193,4.5,Tai nghe bluetooth không dây F9 True wireless ...,không dây wireless có báo kép nhạc giờ giờ . o...
2,2,299461,4.8,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,chuột không dây hàng chính hãng thương hiệu cá...


In [22]:
product_ID= 299461
product= products[products.item_id==product_ID].head(1)

In [23]:
product[["index","item_id","product_content"]]

,index,item_id,product_content
2,2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...


### Chọn Product để thực hiện Recommend

In [24]:
products.loc[products["item_id"]==299461]["product_content"].values[0]

'Chuột Không Dây Logitech M331 Silent - Hàng Chính Hãng THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ phân giải quang học 1000DPI\nModel M331\nLưu ý\nCách kéo dài tuổi thọ pin cho chuột:\nOn/ Off nguồn mỗi khi không sử dụng lâu hoặc hết giờ làm việc\nHãy đảm bảo phần USB nhận tín hiệu (Receiver) gần chuột nhất có thể\nTránh sử dụng bàn chuột màu tối hoặc đục mờ. Mắt cảm ứng là phần tiêu hao năng lượng nhiều nhất của chuột, nó tốn nhiều năng lượng hơn trên bề mặt tối, đục hay gồ ghề\nHãy dùng bàn chuột màu sáng và mượt để tối ưu việc sử dụng năng lượng\nSử dụng pin cùng dung lượng và cùng hãng để tăng hiệu suất của pin\nXuất xứ Trung Quốc\nSKU 3057399746627\nMÔ TẢ SẢN PHẨM\nĐộ phân giải: 1000DPI\n3 nút nhấn cơ bản và 1 cần gạt on/of\nKết nối không dây 2.4Ghz\nKhoảng cách kết nối: 10m\nSử dụng pin AA với cảm biến quang giúp kéo dài tuổi thọ pin\nThiết kế sang trọng\nChuột Không Dây Logitech M331 Silent được thiết kế với các đường cong uốn lượn, bo tròn và ôm sát lòng bàn tay, giúp người s

In [25]:
#sản phẩm đang xem
name_description_pre=product["product_content"].to_string(index=False)

In [26]:
name_description_pre

'Chuột Không Dây Logitech M331 Silent - Hàng Chí...'

In [27]:
def recomender(view_product,dictionary,tfidf,index):
    view_product=view_product.lower().split()
    kw_vector=dictionary.doc2bow(view_product)
    print("View product's vector:")
    print(kw_vector)
    
    #similarity calculation
    sim=index[tfidf[kw_vector]]
    
    list_id=[]
    list_score=[]
    for i in range(len(sim)):
        list_id.append(i)
        list_score.append(sim[i])
        
    df_result=pd.DataFrame({"id":list_id, "score":list_score})
    
    #five highest scores
    five_highest_score=df_result.sort_values(by="score",ascending=False).head()
    print("Five highest scores: ")
    print(five_highest_score)
    print("Ids to list:")
    idToList=list(five_highest_score["id"])
    print(idToList)
    
    products_find=products[products.index.isin(idToList)]
    results=products_find[["index","item_id","product_content"]]
    results=pd.concat([results,five_highest_score],axis=1).sort_values(by="score",ascending=False)
    return results

In [28]:
results=recomender(name_description_pre,dictionary,tfidf,index)

View product's vector:
[(35, 1), (42, 1), (75, 1), (257, 1), (483, 1)]
Five highest scores: 
        id     score
28      28  0.547776
355    355  0.508629
326    326  0.441103
149    149  0.354824
2023  2023  0.354824
Ids to list:
[28, 355, 326, 149, 2023]


### Đề xuất 5 sản phẩm tương tự cho sản phẩm đã chọn

In [29]:
results= results[results.item_id != product_ID]
results

,index,item_id,product_content,id,score
28,28,916784,Chuột Không Dây Logitech M590 Slient - Hàng Ch...,28,0.547776
355,355,515550,Bộ Bàn Phím Và Chuột Không Dây Logitech MK235 ...,355,0.508629
326,326,899588,Chuột Không Dây DareU LM115G - Hàng Chính Hãng...,326,0.441103
149,149,9830444,Chuột Không Dây Logitech M238 Marvel Collectio...,149,0.354824
2023,2023,9830444,Chuột Không Dây Logitech M238 Marvel Collectio...,2023,0.354824


In [81]:
# products.to_csv("pre_products.csv")